# Train Amazon Comprehend Custom Classifier Model

<img src="img/comprehend.png" width="80%" align="left">

## Note that Amazon Comprehend is currently only supported in a subset of regions: 

* US East (N. Virginia), US East (Ohio), US West (Oregon)
* Canada (Central)
* Europe (London), Europe (Ireland), Europe (Frankfurt)
* Asia Pacific (Mumbai), Asia Pacific (Seoul), Asia Pacific (Tokyo), Asia Pacific (Singapore), Asia Pacific (Sydney)

You can check https://aws.amazon.com/about-aws/global-infrastructure/regional-product-services/ for details and updates. 

In [32]:
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

from botocore.config import Config

config = Config(retries={"max_attempts": 10, "mode": "adaptive"})

iam = boto3.client("iam", config=config)
sm = boto3.Session().client(service_name="sagemaker", region_name=region)

### Check if you current regions supports Comprehend

In [33]:
if region in [
    "ap-south-1",
    "eu-west-2",
    "eu-west-1",
    "ap-northeast-2",
    "ap-northeast-1",
    "ca-central-1",
    "ap-southeast-1",
    "ap-southeast-2",
    "eu-central-1",
    "us-east-1",
    "us-east-2",
    "us-west-2",
]:
    print(" [OK] COMPREHEND IS SUPPORTED IN {}".format(region))
    print(" [OK] Please proceed with this notebook.")
else:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print(" [ERROR] COMPREHEND IS NOT YET SUPPORTED IN {}.".format(region))
    print(" [INFO] This is OK. Skip this notebook and continue with the next use case.")
    print(" [INFO] This notebook is not required for the rest of this workshop.")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

 [OK] COMPREHEND IS SUPPORTED IN us-east-1
 [OK] Please proceed with this notebook.


In [34]:
comprehend = boto3.client("comprehend")

### Retrieve S3 location of training data

In [35]:
%store -r comprehend_train_s3_uri

In [36]:
if not comprehend_train_s3_uri:
    print("****************************************************************************************")
    print("**************** PLEASE RE-RUN THE PREVIOUS DATA PREPARATION NOTEBOOK ******************")
    print("**************** THIS NOTEBOOK WILL NOT RUN PROPERLY ***********************************")
    print("****************************************************************************************")

In [37]:
print(comprehend_train_s3_uri)

s3://sagemaker-us-east-1-701096626288/data/amazon_reviews_us_Digital_Software_v1_00_comprehend.csv


In [ ]:
!aws s3 ls $comprehend_train_s3_uri

2023-03-01 02:41:53   13747086 amazon_reviews_us_Digital_Software_v1_00_comprehend.csv


## See our prepared training data which we use as input for Comprehend

In [ ]:
!aws s3 cp $comprehend_train_s3_uri ./tmp/

download: s3://sagemaker-us-east-1-701096626288/data/amazon_reviews_us_Digital_Software_v1_00_comprehend.csv to tmp/amazon_reviews_us_Digital_Software_v1_00_comprehend.csv


In [ ]:
import csv

df = pd.read_csv("./tmp/amazon_reviews_us_Digital_Software_v1_00_comprehend.csv", header=None)
df.head()

,0,1
0,2,Isn't all it claims to be. Grocery lists don't...
1,4,I like buying this product from Amazon each ye...
2,5,Works good in our laptops..
3,3,"As other persons has outlined, the quick books..."
4,4,I have used Norton for quite a few years and a...


# Create Data Access Role for Comprehend

## Create Policy

In [54]:
assume_role_policy_doc = {
    "Version": "2012-10-17",
    "Statement": [
        {"Effect": "Allow", "Principal": {"Service": "comprehend.amazonaws.com"}, "Action": "sts:AssumeRole"}
    ],
}

## Create Role and Attach Policies

In [55]:
iam_comprehend_role_name = "LabRole"

In [56]:
import json
import time

from botocore.exceptions import ClientError

iam_role_comprehend = iam.get_role(RoleName=iam_comprehend_role_name)

print(iam_role_comprehend)

{'Role': {'Path': '/', 'RoleName': 'LabRole', 'RoleId': 'AROA2GPE5RBYNBYSQFVHU', 'Arn': 'arn:aws:iam::701096626288:role/LabRole', 'CreateDate': datetime.datetime(2023, 2, 28, 4, 31, 4, tzinfo=tzlocal()), 'AssumeRolePolicyDocument': {'Version': '2012-10-17', 'Statement': [{'Effect': 'Allow', 'Principal': {'Service': ['elasticbeanstalk.amazonaws.com', 'redshift.amazonaws.com', 'cloudformation.amazonaws.com', 'athena.amazonaws.com', 'elasticmapreduce.amazonaws.com', 'codedeploy.amazonaws.com', 'states.amazonaws.com', 'ec2.application-autoscaling.amazonaws.com', 'sqs.amazonaws.com', 'apigateway.amazonaws.com', 'personalize.amazonaws.com', 'cloudfront.amazonaws.com', 'cloud9.amazonaws.com', 'forecast.amazonaws.com', 'ec2.amazonaws.com', 'batch.amazonaws.com', 'autoscaling.amazonaws.com', 'robomaker.amazonaws.com', 'rds.amazonaws.com', 'backup.amazonaws.com', 'lex.amazonaws.com', 'resource-groups.amazonaws.com', 'servicecatalog.amazonaws.com', 'comprehend.amazonaws.com', 'credentials.iot.ama

In [57]:
comprehend_s3_policy_doc = {
    "Version": "2012-10-17",
    "Statement": [
        {"Action": ["s3:GetObject"], "Resource": ["arn:aws:s3:::{}/*".format(bucket)], "Effect": "Allow"},
        {"Action": ["s3:ListBucket"], "Resource": ["arn:aws:s3:::{}".format(bucket)], "Effect": "Allow"},
        {"Action": ["s3:PutObject"], "Resource": ["arn:aws:s3:::{}/*".format(bucket)], "Effect": "Allow"},
    ],
}

print(comprehend_s3_policy_doc)

{'Version': '2012-10-17', 'Statement': [{'Action': ['s3:GetObject'], 'Resource': ['arn:aws:s3:::sagemaker-us-east-1-701096626288/*'], 'Effect': 'Allow'}, {'Action': ['s3:ListBucket'], 'Resource': ['arn:aws:s3:::sagemaker-us-east-1-701096626288'], 'Effect': 'Allow'}, {'Action': ['s3:PutObject'], 'Resource': ['arn:aws:s3:::sagemaker-us-east-1-701096626288/*'], 'Effect': 'Allow'}]}


# Train the Model

In [58]:
prefix = "models"

s3_output_job = "s3://{}/{}/{}".format(bucket, prefix, "comprehend/output")
print(s3_output_job)

s3://sagemaker-us-east-1-701096626288/models/comprehend/output


In [59]:
iam_role_comprehend_arn = iam_role_comprehend["Role"]["Arn"]

In [60]:
import datetime
import time

timestamp = str(datetime.datetime.now().strftime("%s"))

comprehend_training_job_name = "Amazon-Customer-Reviews-Classifier-{}".format(timestamp)

print(comprehend_training_job_name)

Amazon-Customer-Reviews-Classifier-1677638804


In [61]:
training_job = comprehend.create_document_classifier(
    DocumentClassifierName=comprehend_training_job_name,
    DataAccessRoleArn=iam_role_comprehend_arn,
    InputDataConfig={"S3Uri": comprehend_train_s3_uri},
    OutputDataConfig={"S3Uri": s3_output_job},
    LanguageCode="en",
)

time.sleep(30)

In [62]:
comprehend_training_job_arn = training_job["DocumentClassifierArn"]

print(comprehend_training_job_arn)

arn:aws:comprehend:us-east-1:701096626288:document-classifier/Amazon-Customer-Reviews-Classifier-1677638804


In [63]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/comprehend/v2/home?region={}#classifier-details/{}">Comprehend Training Job</a></b>'.format(
            region, comprehend_training_job_arn
        )
    )
)

# This Next Cell Will Take Some Time
# _Please be patient._

In [ ]:
import time

max_time = time.time() + 3 * 60 * 60  # 3 hours
while time.time() < max_time:
    describe_custom_classifier = comprehend.describe_document_classifier(
        DocumentClassifierArn=comprehend_training_job_arn
    )
    status = describe_custom_classifier["DocumentClassifierProperties"]["Status"]
    print("Custom classifier: {}".format(status))

    if status == "TRAINED" or status == "IN_ERROR":
        print("")
        print("Status {}".format(status))
        print("")
        print(describe_custom_classifier["DocumentClassifierProperties"])
        break

    time.sleep(10)

Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom cla

# _Please Wait Until the ^^ Classifier ^^ is Trained Above._

# [INFO] _Feel free to continue to the next workshop section while this notebook is running._

# Show Results of the Classifier

In [ ]:
print(describe_custom_classifier["DocumentClassifierProperties"])

In [ ]:
model_arn = describe_custom_classifier["DocumentClassifierProperties"]["DocumentClassifierArn"]
print(model_arn)

In [ ]:
import os

# Retrieve the S3URI from the model output and create jobkey variable.
job_output = describe_custom_classifier["DocumentClassifierProperties"]["OutputDataConfig"]["S3Uri"]
print(job_output)

path_prefix = "s3://{}/".format(bucket)

job_key = os.path.relpath(job_output, path_prefix)

print(job_key)

# Download Model Artifacts including Training Metrics

In [ ]:
s3 = boto3.resource("s3")

s3.Bucket(bucket).download_file(job_key, "./output.tar.gz")

In [ ]:
# Unpack the gzip file
!tar xvzf ./output.tar.gz

In [ ]:
import json

with open("./output/confusion_matrix.json") as json_file:
    data = json.load(json_file)
print(json.dumps(data, indent=2, default=str))

In [ ]:
!pip install tabulate

In [ ]:
from IPython.display import HTML, display
import tabulate

table = [
    ["", "1", "2", "3", "4", "5", "(Predicted)"],
    [
        "1",
        data["confusion_matrix"][0][0],
        data["confusion_matrix"][0][1],
        data["confusion_matrix"][0][2],
        data["confusion_matrix"][0][3],
        data["confusion_matrix"][0][4],
    ],
    [
        "2",
        data["confusion_matrix"][1][0],
        data["confusion_matrix"][1][1],
        data["confusion_matrix"][1][2],
        data["confusion_matrix"][1][3],
        data["confusion_matrix"][1][4],
    ],
    [
        "3",
        data["confusion_matrix"][2][0],
        data["confusion_matrix"][2][1],
        data["confusion_matrix"][2][2],
        data["confusion_matrix"][2][3],
        data["confusion_matrix"][2][4],
    ],
    [
        "4",
        data["confusion_matrix"][3][0],
        data["confusion_matrix"][3][1],
        data["confusion_matrix"][3][2],
        data["confusion_matrix"][3][3],
        data["confusion_matrix"][3][4],
    ],
    [
        "5",
        data["confusion_matrix"][4][0],
        data["confusion_matrix"][4][1],
        data["confusion_matrix"][4][2],
        data["confusion_matrix"][4][3],
        data["confusion_matrix"][4][4],
    ],
    ["(Actual)"],
]
display(HTML(tabulate.tabulate(table, tablefmt="html")))

# Deploy Endpoint

In [ ]:
from time import gmtime, strftime, sleep

timestamp_suffix = strftime("%d-%H-%M-%S", gmtime())

comprehend_endpoint_name = "comprehend-inference-ep-" + timestamp_suffix

inference_endpoint_response = comprehend.create_endpoint(
    EndpointName=comprehend_endpoint_name, ModelArn=model_arn, DesiredInferenceUnits=1
)

In [ ]:
comprehend_endpoint_arn = inference_endpoint_response["EndpointArn"]
print(comprehend_endpoint_arn)

# Pass Variables to the Next Notebook(s)

In [ ]:
%store comprehend_training_job_arn

In [ ]:
%store comprehend_endpoint_arn

# Release Resources

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}